In [ ]:
import pandas as pd
import numpy as np
import math
import scipy
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import SimpleImputer

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV

import statsmodels

from statsmodels.tools.eval_measures import rmse
from sklearn.metrics import r2_score,mean_squared_error, f1_score
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

from scipy import stats

In [ ]:
!gdown --id 1TKLIvKFEM8iNVPWd98Dlqr99sfy6wCxC

Downloading...
From: https://drive.google.com/uc?id=1TKLIvKFEM8iNVPWd98Dlqr99sfy6wCxC
To: /content/Kaggle_Sirio_Libanes_ICU_Prediction.csv
100% 2.25M/2.25M [00:00<00:00, 72.5MB/s]


### Preliminaries


In [ ]:
h = pd.read_csv('/content/Kaggle_Sirio_Libanes_ICU_Prediction.csv')

In [ ]:

h.isnull().sum()

PATIENT_VISIT_IDENTIFIER        0
AGE_ABOVE65                     0
AGE_PERCENTIL                   0
GENDER                          0
DISEASE GROUPING 1              5
                             ... 
RESPIRATORY_RATE_DIFF_REL     748
TEMPERATURE_DIFF_REL          694
OXYGEN_SATURATION_DIFF_REL    686
WINDOW                          0
ICU                             0
Length: 231, dtype: int64

In [ ]:
h.dtypes

PATIENT_VISIT_IDENTIFIER        int64
AGE_ABOVE65                     int64
AGE_PERCENTIL                  object
GENDER                          int64
DISEASE GROUPING 1            float64
                               ...   
RESPIRATORY_RATE_DIFF_REL     float64
TEMPERATURE_DIFF_REL          float64
OXYGEN_SATURATION_DIFF_REL    float64
WINDOW                         object
ICU                             int64
Length: 231, dtype: object

In [ ]:
h1 = h.drop(['PATIENT_VISIT_IDENTIFIER','WINDOW'],axis=1)
h1['AGE_PERCENTIL'].replace('th','', regex=True, inplace=True)
h1['AGE_PERCENTIL'].replace('Above 90','90', regex=True, inplace=True)

In [ ]:
h1.iloc[:, 1]


0       60
1       60
2       60
3       60
4       60
        ..
1920    50
1921    50
1922    50
1923    50
1924    50
Name: AGE_PERCENTIL, Length: 1925, dtype: object

In [ ]:
h1_col = h1.columns

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer = imputer.fit(h1)
  
h1_transformed = imputer.transform(h1)

In [ ]:
h1_work = pd.DataFrame(columns=h1_col, data=h1_transformed)

In [ ]:
h1

,AGE_ABOVE65,AGE_PERCENTIL,GENDER,DISEASE GROUPING 1,DISEASE GROUPING 2,DISEASE GROUPING 3,DISEASE GROUPING 4,DISEASE GROUPING 5,DISEASE GROUPING 6,HTN,IMMUNOCOMPROMISED,OTHER,ALBUMIN_MEDIAN,ALBUMIN_MEAN,ALBUMIN_MIN,ALBUMIN_MAX,ALBUMIN_DIFF,BE_ARTERIAL_MEDIAN,BE_ARTERIAL_MEAN,BE_ARTERIAL_MIN,BE_ARTERIAL_MAX,BE_ARTERIAL_DIFF,BE_VENOUS_MEDIAN,BE_VENOUS_MEAN,BE_VENOUS_MIN,BE_VENOUS_MAX,BE_VENOUS_DIFF,BIC_ARTERIAL_MEDIAN,BIC_ARTERIAL_MEAN,BIC_ARTERIAL_MIN,BIC_ARTERIAL_MAX,BIC_ARTERIAL_DIFF,BIC_VENOUS_MEDIAN,BIC_VENOUS_MEAN,BIC_VENOUS_MIN,BIC_VENOUS_MAX,BIC_VENOUS_DIFF,BILLIRUBIN_MEDIAN,BILLIRUBIN_MEAN,BILLIRUBIN_MIN,...,DIMER_MIN,DIMER_MAX,DIMER_DIFF,BLOODPRESSURE_DIASTOLIC_MEAN,BLOODPRESSURE_SISTOLIC_MEAN,HEART_RATE_MEAN,RESPIRATORY_RATE_MEAN,TEMPERATURE_MEAN,OXYGEN_SATURATION_MEAN,BLOODPRESSURE_DIASTOLIC_MEDIAN,BLOODPRESSURE_SISTOLIC_MEDIAN,HEART_RATE_MEDIAN,RESPIRATORY_RATE_MEDIAN,TEMPERATURE_MEDIAN,OXYGEN_SATURATION_MEDIAN,BLOODPRESSURE_DIASTOLIC_MIN,BLOODPRESSURE_SISTOLIC_MIN,HEART_RATE_MIN,RESPIRATORY_RATE_MIN,TEMPERATURE_MIN,OXYGEN_SATURATION_MIN,BLOODPRESSURE_DIASTOLIC_MAX,BLOODPRESSURE_SISTOLIC_MAX,HEART_RATE_MAX,RESPIRATORY_RATE_MAX,TEMPERATURE_MAX,OXYGEN_SATURATION_MAX,BLOODPRESSURE_DIASTOLIC_DIFF,BLOODPRESSURE_SISTOLIC_DIFF,HEART_RATE_DIFF,RESPIRATORY_RATE_DIFF,TEMPERATURE_DIFF,OXYGEN_SATURATION_DIFF,BLOODPRESSURE_DIASTOLIC_DIFF_REL,BLOODPRESSURE_SISTOLIC_DIFF_REL,HEART_RATE_DIFF_REL,RESPIRATORY_RATE_DIFF_REL,TEMPERATURE_DIFF_REL,OXYGEN_SATURATION_DIFF_REL,ICU
0,1,60,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.086420,-0.230769,-0.283019,-0.593220,-0.285714,0.736842,0.086420,-0.230769,-0.283019,-0.586207,-0.285714,0.736842,0.237113,0.0000,-0.162393,-0.500000,0.208791,0.898990,-0.247863,-0.459459,-0.432836,-0.636364,-0.420290,0.736842,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0
1,1,60,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.333333,-0.230769,-0.132075,-0.593220,0.535714,0.578947,0.333333,-0.230769,-0.132075,-0.586207,0.535714,0.578947,0.443299,0.0000,-0.025641,-0.500000,0.714286,0.838384,-0.076923,-0.459459,-0.313433,-0.636364,0.246377,0.578947,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0
2,1,60,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.605263,0.605263,0.605263,0.605263,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.938950,-0.938950,-0.938950,...,-0.994912,-0.994912,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,1,60,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.107143,0.736842,NaN,NaN,NaN,NaN,-0.107143,0.736842,NaN,NaN,NaN,NaN,0.318681,0.898990,NaN,NaN,NaN,NaN,-0.275362,0.736842,NaN,NaN,NaN,NaN,-1.000000,-1.000000,NaN,NaN,NaN,NaN,-1.000000,-1.000000,0
4,1,60,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.000000,0.000000,0.000000,0.000000,-1.0,-0.871658,-0.871658,-0.871658,-0.871658,-1.0,-0.863874,-0.863874,-0.863874,-0.863874,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.414634,-0.414634,-0.414634,-0.414634,-1.0,-0.979069,-0.979069,-0.979069,...,-0.996762,-0.996762,-1.0,-0.243021,-0.338537,-0.213031,-0.317859,0.033779,0.665932,-0.283951,-0.376923,-0.188679,-0.379310,0.035714,0.631579,-0.340206,-0.4875,-0.572650,-0.857143,0.098901,0.797980,-0.076923,0.286486,0.298507,0.272727,0.362319,0.947368,-0.339130,0.325153,0.114504,0.176471,-0.238095,-0.818

In [ ]:
for column in np.array(h1_work.columns) :
    if h1_work.var()[column] == 0:
        h1_work = h1_work.drop(column,axis=1)

In [ ]:
h1_work

,AGE_ABOVE65,AGE_PERCENTIL,GENDER,DISEASE GROUPING 1,DISEASE GROUPING 2,DISEASE GROUPING 3,DISEASE GROUPING 4,DISEASE GROUPING 5,DISEASE GROUPING 6,HTN,IMMUNOCOMPROMISED,OTHER,ALBUMIN_MEDIAN,ALBUMIN_MEAN,ALBUMIN_MIN,ALBUMIN_MAX,BE_ARTERIAL_MEDIAN,BE_ARTERIAL_MEAN,BE_ARTERIAL_MIN,BE_ARTERIAL_MAX,BE_VENOUS_MEDIAN,BE_VENOUS_MEAN,BE_VENOUS_MIN,BE_VENOUS_MAX,BIC_ARTERIAL_MEDIAN,BIC_ARTERIAL_MEAN,BIC_ARTERIAL_MIN,BIC_ARTERIAL_MAX,BIC_VENOUS_MEDIAN,BIC_VENOUS_MEAN,BIC_VENOUS_MIN,BIC_VENOUS_MAX,BILLIRUBIN_MEDIAN,BILLIRUBIN_MEAN,BILLIRUBIN_MIN,BILLIRUBIN_MAX,BLAST_MEDIAN,BLAST_MEAN,BLAST_MIN,BLAST_MAX,...,DIMER_MEAN,DIMER_MIN,DIMER_MAX,BLOODPRESSURE_DIASTOLIC_MEAN,BLOODPRESSURE_SISTOLIC_MEAN,HEART_RATE_MEAN,RESPIRATORY_RATE_MEAN,TEMPERATURE_MEAN,OXYGEN_SATURATION_MEAN,BLOODPRESSURE_DIASTOLIC_MEDIAN,BLOODPRESSURE_SISTOLIC_MEDIAN,HEART_RATE_MEDIAN,RESPIRATORY_RATE_MEDIAN,TEMPERATURE_MEDIAN,OXYGEN_SATURATION_MEDIAN,BLOODPRESSURE_DIASTOLIC_MIN,BLOODPRESSURE_SISTOLIC_MIN,HEART_RATE_MIN,RESPIRATORY_RATE_MIN,TEMPERATURE_MIN,OXYGEN_SATURATION_MIN,BLOODPRESSURE_DIASTOLIC_MAX,BLOODPRESSURE_SISTOLIC_MAX,HEART_RATE_MAX,RESPIRATORY_RATE_MAX,TEMPERATURE_MAX,OXYGEN_SATURATION_MAX,BLOODPRESSURE_DIASTOLIC_DIFF,BLOODPRESSURE_SISTOLIC_DIFF,HEART_RATE_DIFF,RESPIRATORY_RATE_DIFF,TEMPERATURE_DIFF,OXYGEN_SATURATION_DIFF,BLOODPRESSURE_DIASTOLIC_DIFF_REL,BLOODPRESSURE_SISTOLIC_DIFF_REL,HEART_RATE_DIFF_REL,RESPIRATORY_RATE_DIFF_REL,TEMPERATURE_DIFF_REL,OXYGEN_SATURATION_DIFF_REL,ICU
0,1.0,60.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.528527,0.528527,0.528527,0.528527,-0.963433,-0.963433,-0.963433,-0.963433,-0.931121,-0.931121,-0.931121,-0.931121,-0.310924,-0.310924,-0.310924,-0.310924,-0.311845,-0.311845,-0.311845,-0.311845,-0.945928,-0.945928,-0.945928,-0.945928,-0.994424,-0.994424,-0.994424,-0.994424,...,-0.954177,-0.954177,-0.954177,0.086420,-0.230769,-0.283019,-0.593220,-0.285714,0.736842,0.086420,-0.230769,-0.283019,-0.586207,-0.285714,0.736842,0.237113,0.000000,-0.162393,-0.500000,0.208791,0.898990,-0.247863,-0.459459,-0.432836,-0.636364,-0.420290,0.736842,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.0
1,1.0,60.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.528527,0.528527,0.528527,0.528527,-0.963433,-0.963433,-0.963433,-0.963433,-0.931121,-0.931121,-0.931121,-0.931121,-0.310924,-0.310924,-0.310924,-0.310924,-0.311845,-0.311845,-0.311845,-0.311845,-0.945928,-0.945928,-0.945928,-0.945928,-0.994424,-0.994424,-0.994424,-0.994424,...,-0.954177,-0.954177,-0.954177,0.333333,-0.230769,-0.132075,-0.593220,0.535714,0.578947,0.333333,-0.230769,-0.132075,-0.586207,0.535714,0.578947,0.443299,0.000000,-0.025641,-0.500000,0.714286,0.838384,-0.076923,-0.459459,-0.313433,-0.636364,0.246377,0.578947,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.0
2,1.0,60.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.605263,0.605263,0.605263,0.605263,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-0.317073,-0.317073,-0.317073,-0.317073,-0.317073,-0.317073,-0.317073,-0.317073,-0.938950,-0.938950,-0.938950,-0.938950,-1.000000,-1.000000,-1.000000,-1.000000,...,-0.994912,-0.994912,-0.994912,-0.093631,-0.332600,-0.264701,-0.438754,0.066893,0.743077,-0.097790,-0.338468,-0.268632,-0.435121,0.063798,0.748588,-0.040855,-0.207813,-0.264999,-0.483129,0.326823,0.817565,-0.235001,-0.399582,-0.282029,-0.316753,0.014964,0.818593,-0.752454,-0.728053,-0.754100,-0.703683,-0.770338,-0.887196,-0.786997,-0.715950,-0.817800,-0.719147,-0.771327,-0.886982,0.0
3,1.0,60.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.528527,0.528527,0.528527,0.528527,-0.963433,-0.963433,-0.963433,-0.963433,-0.931121,-0.931121,-0.931121,-0.931121,-0.310924,-0.310924,-0.310924,-0.310924,-0.311845,-0.311845,-0.311845,-0.311845,-0.945928,-0.945928,-0.945928,-0.945928,-0.994424,-0.994424,-0.994424,-0.994424,...,-0.954177,-0.954177,-0.954177,-0.093631,-0.332600,

In [ ]:
def check(str1, str2):
  if str1.find(str2) == -1 :
    return False
  else :
    return True

In [ ]:
h2_work = h1_work
for column in np.array(h1_work.columns):
    if check(column,'MEDIAN') or check(column,'MIN') or check(column,'MAX'):
        h2_work = h2_work.drop(column,axis=1)

In [ ]:
display(h2_work.iloc[:,[0,1,2,3,4,-1]])

,AGE_ABOVE65,AGE_PERCENTIL,GENDER,DISEASE GROUPING 1,DISEASE GROUPING 2,ICU
0,1.0,60.0,0.0,0.0,0.0,0.0
1,1.0,60.0,0.0,0.0,0.0,0.0
2,1.0,60.0,0.0,0.0,0.0,0.0
3,1.0,60.0,0.0,0.0,0.0,0.0
4,1.0,60.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...
1920,0.0,50.0,1.0,0.0,0.0,0.0
1921,0.0,50.0,1.0,0.0,0.0,0.0
1922,0.0,50.0,1.0,0.0,0.0,0.0
1923,0.0,50.0,1.0,0.0,0.0,0.0


# **Hypothesis Testing - $\chi^2$ Contingency Test**

**1) Age & ICU Admission**

With ${\alpha}$ = 0.05 <br>
We have Null Hypothesis, ${H_0}$ : No relation between age percentile and ICU admission <br>
We have Alternate Hypothesis, ${H_1}$ : Certain relation exists between age percentile and ICU admission <br>

In [ ]:
from scipy.stats import chi2_contingency #Here chi_2contingency is a function of the scipy.stats module
  
# defining the table having ages and ICU admission
stat, p, dof, expected = chi2_contingency(pd.crosstab(h1['AGE_PERCENTIL'],h1['ICU']))


#Interpretation:
# stat gives the statistics of df_chi; p gives the p value; dof is the degree of freedom; expected gives the expected value of each data point;
  
# interpret p-value
alpha = 0.05
print("p value is " + str(p))
if p <= alpha:
    print('Dependent (reject H0)')
else:
    print('Independent (H0 holds true)')

p value is 3.441686189606756e-21
Dependent (reject H0)


**2) Blood Pressure & ICU Admission**

With ${\alpha}$ = 0.05 <br>
We have Null Hypothesis, ${H_0}$ : No relation between blood pressure and ICU admission <br>
We have Alternate Hypothesis, ${H_1}$ : Certain relation exists between blood pressure and ICU admission <br>

In [ ]:
from scipy.stats import chi2_contingency
  

stat, p, dof, expected = chi2_contingency(pd.crosstab(h1['BLOODPRESSURE_DIASTOLIC_MEAN'],h1['ICU']))



alpha = 0.05
print("p value is " + str(p))
if p <= alpha:
    print('Dependent (reject H0)')
else:
    print('Independent (H0 holds true)')

p value is 2.758059578137831e-09
Dependent (reject H0)


**3) Oxygen Saturation Level & ICU Admission**

With ${\alpha}$ = 0.05 <br>
We have Null Hypothesis, ${H_0}$ : No relation between oxygen saturation level and ICU admission <br>
We have Alternate Hypothesis, ${H_1}$ : Certain relation exists between oxygen saturation level and ICU admission <br>

In [ ]:
from scipy.stats import chi2_contingency
  

stat, p, dof, expected = chi2_contingency(pd.crosstab(h1['OXYGEN_SATURATION_MIN'],h1['ICU']))



alpha = 0.05
print("p value is " + str(p))
if p <= alpha:
    print('Dependent (reject H0)')
else:
    print('Independent (H0 holds true)')

p value is 2.212150440289027e-15
Dependent (reject H0)


**4) Billirubin Concentration(pigment) & ICU Admission**

With ${\alpha}$ = 0.05 <br>
We have Null Hypothesis, ${H_0}$ : No relation between Billirubin Concentration and ICU admission <br>
We have Alternate Hypothesis, ${H_1}$ : Certain relation exists between Billirubin concentration and ICU admission <br>

In [ ]:
from scipy.stats import chi2_contingency
  

stat, p, dof, expected = chi2_contingency(pd.crosstab(h1['BILLIRUBIN_MEAN'],h1['ICU']))


alpha = 0.05
print("p value is " + str(p))
if p <= alpha:
    print('Dependent (reject H0)')
else:
    print('Independent (H0 holds true)')

p value is 0.20537769804572933
Independent (H0 holds true)


#***Binary Classification***

# **1) Logistic Regression**

In [ ]:
h2_work.iloc[:,:-1]

,AGE_ABOVE65,AGE_PERCENTIL,GENDER,DISEASE GROUPING 1,DISEASE GROUPING 2,DISEASE GROUPING 3,DISEASE GROUPING 4,DISEASE GROUPING 5,DISEASE GROUPING 6,HTN,IMMUNOCOMPROMISED,OTHER,BE_ARTERIAL_MEAN,BE_VENOUS_MEAN,BIC_ARTERIAL_MEAN,BIC_VENOUS_MEAN,BILLIRUBIN_MEAN,BLAST_MEAN,CALCIUM_MEAN,CREATININ_MEAN,FFA_MEAN,GGT_MEAN,GLUCOSE_MEAN,HEMATOCRITE_MEAN,HEMOGLOBIN_MEAN,INR_MEAN,LACTATE_MEAN,LEUKOCYTES_MEAN,LINFOCITOS_MEAN,NEUTROPHILES_MEAN,P02_ARTERIAL_MEAN,P02_VENOUS_MEAN,PC02_ARTERIAL_MEAN,PC02_VENOUS_MEAN,PCR_MEAN,PH_ARTERIAL_MEAN,PH_VENOUS_MEAN,PLATELETS_MEAN,POTASSIUM_MEAN,SAT02_ARTERIAL_MEAN,SAT02_VENOUS_MEAN,SODIUM_MEAN,TGO_MEAN,TGP_MEAN,TTPA_MEAN,UREA_MEAN,DIMER_MEAN,BLOODPRESSURE_DIASTOLIC_MEAN,BLOODPRESSURE_SISTOLIC_MEAN,HEART_RATE_MEAN,RESPIRATORY_RATE_MEAN,TEMPERATURE_MEAN,OXYGEN_SATURATION_MEAN,BLOODPRESSURE_DIASTOLIC_DIFF,BLOODPRESSURE_SISTOLIC_DIFF,HEART_RATE_DIFF,RESPIRATORY_RATE_DIFF,TEMPERATURE_DIFF,OXYGEN_SATURATION_DIFF,BLOODPRESSURE_DIASTOLIC_DIFF_REL,BLOODPRESSURE_SISTOLIC_DIFF_REL,HEART_RATE_DIFF_REL,RESPIRATORY_RATE_DIFF_REL,TEMPERATURE_DIFF_REL,OXYGEN_SATURATION_DIFF_REL
0,1.0,60.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,-0.963433,-0.931121,-0.310924,-0.311845,-0.945928,-0.994424,0.330359,-0.891078,-0.723217,-0.920403,-0.861694,-0.160818,-0.202472,-0.936950,0.267131,-0.741266,-0.710390,-0.812662,-0.175886,-0.675342,-0.777664,-0.755797,-0.845570,0.236997,0.369007,-0.414479,-0.525624,0.914277,0.331965,-0.053060,-0.991054,-0.982156,-0.822280,-0.830181,-0.954177,0.086420,-0.230769,-0.283019,-0.593220,-0.285714,0.736842,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
1,1.0,60.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,-0.963433,-0.931121,-0.310924,-0.311845,-0.945928,-0.994424,0.330359,-0.891078,-0.723217,-0.920403,-0.861694,-0.160818,-0.202472,-0.936950,0.267131,-0.741266,-0.710390,-0.812662,-0.175886,-0.675342,-0.777664,-0.755797,-0.845570,0.236997,0.369007,-0.414479,-0.525624,0.914277,0.331965,-0.053060,-0.991054,-0.982156,-0.822280,-0.830181,-0.954177,0.333333,-0.230769,-0.132075,-0.593220,0.535714,0.578947,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
2,1.0,60.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,-1.000000,-1.000000,-0.317073,-0.317073,-0.938950,-1.000000,0.183673,-0.868365,-0.742004,-0.945093,-0.891993,0.090147,0.109756,-0.932246,1.000000,-0.835844,-0.914938,-0.868747,-0.170732,-0.704142,-0.779310,-0.754601,-0.875236,0.234043,0.363636,-0.540721,-0.518519,0.939394,0.345679,-0.028571,-0.997201,-0.990854,-0.825613,-0.836145,-0.994912,-0.093631,-0.332600,-0.264701,-0.438754,0.066893,0.743077,-0.752454,-0.728053,-0.754100,-0.703683,-0.770338,-0.887196,-0.786997,-0.715950,-0.817800,-0.719147,-0.771327,-0.886982
3,1.0,60.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,-0.963433,-0.931121,-0.310924,-0.311845,-0.945928,-0.994424,0.330359,-0.891078,-0.723217,-0.920403,-0.861694,-0.160818,-0.202472,-0.936950,0.267131,-0.741266,-0.710390,-0.812662,-0.175886,-0.675342,-0.777664,-0.755797,-0.845570,0.236997,0.369007,-0.414479,-0.525624,0.914277,0.331965,-0.053060,-0.991054,-0.982156,-0.822280,-0.830181,-0.954177,-0.093631,-0.332600,-0.264701,-0.438754,-0.107143,0.736842,-0.752454,-0.728053,-0.754100,-0.703683,-1.000000,-1.000000,-0.786997,-0.715950,-0.817800,-0.719147,-1.000000,-1.000000
4,1.0,60.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,-0.871658,-0.863874,-0.317073,-0.414634,-0.979069,-1.000000,0.326531,-0.926398,-0.859275,-0.669393,-0.891993,-0.320755,-0.353659,-0.979925,-0.963023,-0.762843,-0.643154,-0.868747,-0.365854,-0.230769,-0.875862,-0.815951,-1.000000,0.574468,0.393939,-0.471295,-0.666667,0.848485,0.925926,0.142857,-0.999067,-0.983994,-0.846633,-0.836145,-0.996762,-0.243021,-0.338537,-0.213031,-0.317859,0.033779,0.665932,-0.339130,0.325153,0.114504,0.176471,-0.238095,-0.818182,-0.389967,0.407558,-0.230462,0.096774,-0.242282,-0.814433
...,...,...,...,...,...,...,...,...,...,

In [ ]:
train_split = 0.6
valid_split = 0.8
iterations = 1000

x_train, x_test, y_train, y_test = train_test_split(h2_work.iloc[:,:-1], h2_work['ICU'], test_size=0.2)



In [ ]:
clf1 = LogisticRegression(random_state=0, max_iter=iterations).fit(x_train, y_train)

In [ ]:
logreg_accuracy=clf1.score(x_test,y_test)
logreg_accuracy

0.8519480519480519

In [ ]:

logreg_recall = recall_score(y_test, clf1.predict(x_test))
logreg_recall

0.5445544554455446

In [ ]:
logreg_f1_score = f1_score(y_test, clf1.predict(x_test))
logreg_f1_score

0.6586826347305389

In [ ]:
print("Coefficients of the Logistic Regression Model are-")
clf1.coef_[0]
#printing coefficients of various features in the logistic regression model

Coefficients of the Logistic Regression Model are-


array([-0.23140879,  0.02013477, -0.50454527, -0.40725598,  0.55404775,
       -0.06353858,  1.03685182, -0.0836059 , -0.28730583,  0.78743704,
       -0.48567759, -1.13783228,  0.57106433,  0.17149503, -0.32263772,
       -0.60131411, -0.12711261, -0.06509433, -0.34594342, -0.20587453,
        0.29125906, -0.13848459, -1.08984933, -0.47632197, -0.04294089,
        0.07310609, -0.67076032,  0.50888493,  0.18039297,  0.49390937,
       -0.02678969, -0.1038809 , -0.02034583, -0.13770324, -0.12410633,
       -0.59286287, -0.54385587,  1.16174904,  0.6653266 , -0.33876512,
        0.2217047 ,  0.83306743, -0.01943724,  0.05175853,  0.42040261,
        0.04790881, -0.43223955, -2.80452201,  1.67469904,  0.50266097,
        3.41417059, -1.07887093,  1.32673842,  0.03559897,  0.13161848,
       -0.10499881,  1.30679119, -0.83867839, -0.05589024, -0.02488367,
        0.13928127, -0.11291193,  2.20439273, -0.84168983,  0.06304497])

In [ ]:
lst=[abs(i) for i in clf1.coef_[0]]
min=np.argsort(lst)  # list of sorted indexes of columns having highest weightage in the logistic regression model
lmax=lmin[::-1]

In [ ]:
lmax

array([50, 47, 62, 48, 52, 56, 37, 11, 22, 51,  6, 63, 57, 41,  9, 26, 38,
       15, 35, 12,  4, 36, 27,  2, 49, 29, 10, 23, 46, 44,  3, 18, 39, 14,
       20,  8,  0, 40, 19, 28, 13, 60, 21, 33, 54, 16, 34, 61, 55, 31,  7,
       25, 17,  5, 64, 58, 43, 45, 24, 53, 30, 59, 32,  1, 42])

In [ ]:
list_cols=list(h.columns)  # creating list of columns of dataframe

In [ ]:
for i in range(4):
  fig=plt.figure()
  sns.regplot(y='ICU', x=list_cols[lmax[i]], data=h, marker='x', scatter_kws={"color": "red"}, line_kws={"color": "blue"})
  plt.plot()  # plotting regression plot for most weighed features in linear regression 

In [ ]:
h2_work.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1925 entries, 0 to 1924
Data columns (total 66 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   AGE_ABOVE65                       1925 non-null   float64
 1   AGE_PERCENTIL                     1925 non-null   float64
 2   GENDER                            1925 non-null   float64
 3   DISEASE GROUPING 1                1925 non-null   float64
 4   DISEASE GROUPING 2                1925 non-null   float64
 5   DISEASE GROUPING 3                1925 non-null   float64
 6   DISEASE GROUPING 4                1925 non-null   float64
 7   DISEASE GROUPING 5                1925 non-null   float64
 8   DISEASE GROUPING 6                1925 non-null   float64
 9   HTN                               1925 non-null   float64
 10  IMMUNOCOMPROMISED                 1925 non-null   float64
 11  OTHER                             1925 non-null   float64
 12  BE_ART

In [ ]:
y=h2_work['ICU']
df=h2_work.drop(['ICU'], axis=1)

In [ ]:
df

,AGE_ABOVE65,AGE_PERCENTIL,GENDER,DISEASE GROUPING 1,DISEASE GROUPING 2,DISEASE GROUPING 3,DISEASE GROUPING 4,DISEASE GROUPING 5,DISEASE GROUPING 6,HTN,IMMUNOCOMPROMISED,OTHER,BE_ARTERIAL_MEAN,BE_VENOUS_MEAN,BIC_ARTERIAL_MEAN,BIC_VENOUS_MEAN,BILLIRUBIN_MEAN,BLAST_MEAN,CALCIUM_MEAN,CREATININ_MEAN,FFA_MEAN,GGT_MEAN,GLUCOSE_MEAN,HEMATOCRITE_MEAN,HEMOGLOBIN_MEAN,INR_MEAN,LACTATE_MEAN,LEUKOCYTES_MEAN,LINFOCITOS_MEAN,NEUTROPHILES_MEAN,P02_ARTERIAL_MEAN,P02_VENOUS_MEAN,PC02_ARTERIAL_MEAN,PC02_VENOUS_MEAN,PCR_MEAN,PH_ARTERIAL_MEAN,PH_VENOUS_MEAN,PLATELETS_MEAN,POTASSIUM_MEAN,SAT02_ARTERIAL_MEAN,SAT02_VENOUS_MEAN,SODIUM_MEAN,TGO_MEAN,TGP_MEAN,TTPA_MEAN,UREA_MEAN,DIMER_MEAN,BLOODPRESSURE_DIASTOLIC_MEAN,BLOODPRESSURE_SISTOLIC_MEAN,HEART_RATE_MEAN,RESPIRATORY_RATE_MEAN,TEMPERATURE_MEAN,OXYGEN_SATURATION_MEAN,BLOODPRESSURE_DIASTOLIC_DIFF,BLOODPRESSURE_SISTOLIC_DIFF,HEART_RATE_DIFF,RESPIRATORY_RATE_DIFF,TEMPERATURE_DIFF,OXYGEN_SATURATION_DIFF,BLOODPRESSURE_DIASTOLIC_DIFF_REL,BLOODPRESSURE_SISTOLIC_DIFF_REL,HEART_RATE_DIFF_REL,RESPIRATORY_RATE_DIFF_REL,TEMPERATURE_DIFF_REL,OXYGEN_SATURATION_DIFF_REL
0,1.0,60.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,-0.963433,-0.931121,-0.310924,-0.311845,-0.945928,-0.994424,0.330359,-0.891078,-0.723217,-0.920403,-0.861694,-0.160818,-0.202472,-0.936950,0.267131,-0.741266,-0.710390,-0.812662,-0.175886,-0.675342,-0.777664,-0.755797,-0.845570,0.236997,0.369007,-0.414479,-0.525624,0.914277,0.331965,-0.053060,-0.991054,-0.982156,-0.822280,-0.830181,-0.954177,0.086420,-0.230769,-0.283019,-0.593220,-0.285714,0.736842,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
1,1.0,60.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,-0.963433,-0.931121,-0.310924,-0.311845,-0.945928,-0.994424,0.330359,-0.891078,-0.723217,-0.920403,-0.861694,-0.160818,-0.202472,-0.936950,0.267131,-0.741266,-0.710390,-0.812662,-0.175886,-0.675342,-0.777664,-0.755797,-0.845570,0.236997,0.369007,-0.414479,-0.525624,0.914277,0.331965,-0.053060,-0.991054,-0.982156,-0.822280,-0.830181,-0.954177,0.333333,-0.230769,-0.132075,-0.593220,0.535714,0.578947,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
2,1.0,60.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,-1.000000,-1.000000,-0.317073,-0.317073,-0.938950,-1.000000,0.183673,-0.868365,-0.742004,-0.945093,-0.891993,0.090147,0.109756,-0.932246,1.000000,-0.835844,-0.914938,-0.868747,-0.170732,-0.704142,-0.779310,-0.754601,-0.875236,0.234043,0.363636,-0.540721,-0.518519,0.939394,0.345679,-0.028571,-0.997201,-0.990854,-0.825613,-0.836145,-0.994912,-0.093631,-0.332600,-0.264701,-0.438754,0.066893,0.743077,-0.752454,-0.728053,-0.754100,-0.703683,-0.770338,-0.887196,-0.786997,-0.715950,-0.817800,-0.719147,-0.771327,-0.886982
3,1.0,60.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,-0.963433,-0.931121,-0.310924,-0.311845,-0.945928,-0.994424,0.330359,-0.891078,-0.723217,-0.920403,-0.861694,-0.160818,-0.202472,-0.936950,0.267131,-0.741266,-0.710390,-0.812662,-0.175886,-0.675342,-0.777664,-0.755797,-0.845570,0.236997,0.369007,-0.414479,-0.525624,0.914277,0.331965,-0.053060,-0.991054,-0.982156,-0.822280,-0.830181,-0.954177,-0.093631,-0.332600,-0.264701,-0.438754,-0.107143,0.736842,-0.752454,-0.728053,-0.754100,-0.703683,-1.000000,-1.000000,-0.786997,-0.715950,-0.817800,-0.719147,-1.000000,-1.000000
4,1.0,60.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,-0.871658,-0.863874,-0.317073,-0.414634,-0.979069,-1.000000,0.326531,-0.926398,-0.859275,-0.669393,-0.891993,-0.320755,-0.353659,-0.979925,-0.963023,-0.762843,-0.643154,-0.868747,-0.365854,-0.230769,-0.875862,-0.815951,-1.000000,0.574468,0.393939,-0.471295,-0.666667,0.848485,0.925926,0.142857,-0.999067,-0.983994,-0.846633,-0.836145,-0.996762,-0.243021,-0.338537,-0.213031,-0.317859,0.033779,0.665932,-0.339130,0.325153,0.114504,0.176471,-0.238095,-0.818182,-0.389967,0.407558,-0.230462,0.096774,-0.242282,-0.814433
...,...,...,...,...,...,...,...,...,...,

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1925 entries, 0 to 1924
Data columns (total 65 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   AGE_ABOVE65                       1925 non-null   float64
 1   AGE_PERCENTIL                     1925 non-null   float64
 2   GENDER                            1925 non-null   float64
 3   DISEASE GROUPING 1                1925 non-null   float64
 4   DISEASE GROUPING 2                1925 non-null   float64
 5   DISEASE GROUPING 3                1925 non-null   float64
 6   DISEASE GROUPING 4                1925 non-null   float64
 7   DISEASE GROUPING 5                1925 non-null   float64
 8   DISEASE GROUPING 6                1925 non-null   float64
 9   HTN                               1925 non-null   float64
 10  IMMUNOCOMPROMISED                 1925 non-null   float64
 11  OTHER                             1925 non-null   float64
 12  BE_ART

In [ ]:
df.shape

(1925, 65)

Hence we have 1925 records across 63 features

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df, y, test_size=0.2)

In [ ]:
RFm_par = {'n_estimators' : [5,10,15, 20,30]}
SVMm_par = {'C': [0.1, 1, 10], 'gamma': [0.1, 0.01, 0.001,], 'kernel': ['rbf']} 
MLPm_nn_par = {'hidden_layer_sizes' : [(2,), (4,), (5,),(10,)], 'alpha' : [0.001, 0.1, 1]}

In [ ]:
scores=pd.DataFrame(columns=['Classification Algorithm', 'F1 Score', 'Accuracy'])

## **2) SVM Classifier**

In [ ]:
param_grid = { 'criterion' : ['gini', 'entropy'],
              "max_features": np.arange(1, 9),
              "min_samples_leaf": np.arange(1, 9),
    'max_depth' : [2,4,6,8,10,12]}

In [ ]:
grid = GridSearchCV(SVC(), SVMm_par,scoring='accuracy', cv=5, n_jobs=-1) 
grid.fit(x_train,y_train) 

GridSearchCV(cv=5, estimator=SVC(), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10], 'gamma': [0.1, 0.01, 0.001],
                         'kernel': ['rbf']},
             scoring='accuracy')

In [ ]:
grid.best_estimator_

SVC(C=10, gamma=0.1)

In [ ]:
grid.best_params_

{'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}

In [ ]:
svm_pred=grid.predict(x_test)
f1_score_svm=f1_score(y_test, svm_pred, average='weighted')
f1_score_svm

0.8276176333970356

In [ ]:
accuracy_svm=grid.score(x_test, y_test)
accuracy_svm

0.8363636363636363

In [ ]:
scores=scores.append({'Classification Algorithm':'SVM Classifier', 'F1 Score':f1_score_svm, 'Accuracy':accuracy_svm}, ignore_index=True)

#**3) MLP Classifier**


In [ ]:
mlp=GridSearchCV(MLPClassifier(max_iter=100), MLPm_nn_par, scoring='accuracy', cv=5)
# gridsearching for best parameters for Multilayer Perceptron Classifier
mlp.fit(x_train, y_train)

GridSearchCV(cv=5, estimator=MLPClassifier(max_iter=100),
             param_grid={'alpha': [0.001, 0.1, 1],
                         'hidden_layer_sizes': [(2,), (4,), (5,), (10,)]},
             scoring='accuracy')

In [ ]:
mlp.best_score_   # cross vaildated score of best estimator

0.8233766233766234

In [ ]:
mlp.best_params_
#dictionary of best parameters

{'alpha': 0.001, 'hidden_layer_sizes': (10,)}

In [ ]:
y_pred_mlpc_m=mlp.predict(x_test)

In [ ]:
f1_score_mlp = (f1_score(y_test, y_pred_mlpc_m,average='weighted'))
f1_score_mlp   # printing F1 score 

0.7916053092912596

In [ ]:
accuracy_mlp= (mlp.score(x_test, y_test))
accuracy_mlp  # printing accuracy of the model

0.8155844155844156

In [ ]:
scores=scores.append({'Classification Algorithm':'MLP Classifier', 'F1 Score':f1_score_mlp, 'Accuracy':accuracy_mlp}, ignore_index=True)

#**4) Ensemble Learning Methods**

**a)Bagging Ensemble Learning - Random Forest Classifier**

In [ ]:
RFm = GridSearchCV(RandomForestClassifier(), RFm_par, scoring='neg_mean_squared_error',cv=5) 
#cv here is the cross validation
RFm.fit(x_train, y_train)
RFm.best_score_  # cross validated score

-0.1318181818181818

In [ ]:
RFm.best_params_

{'n_estimators': 30}

In [ ]:
y_pred=RFm.predict(x_test)

In [ ]:
f1_rf_m= (f1_score(y_test, y_pred,average='weighted'))
f1_rf_m

0.86455463728191

In [ ]:
accuracy_rf=accuracy_score(y_test, y_pred)
accuracy_rf

0.8701298701298701

In [ ]:
scores=scores.append({'Classification Algorithm':'Random Forest Classifier', 'F1 Score':f1_rf_m, 'Accuracy':accuracy_rf}, ignore_index=True)

**b)Boosting Ensemble Learning - Gradient Boosting Machines (GBM)**

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gbm_model=GradientBoostingClassifier() #making an instance of the class

#Hyperparameter Tuning & fitting
param_grid={
    'learning_rate':[0.05,0.1,0.2],
    'n_estimators':[50,75,100,125]
}
gbm = GridSearchCV(gbm_model,param_grid,scoring='accuracy',cv=5) 
gbm.fit(x_train,y_train)
y_pred=gbm.predict(x_test)

In [ ]:
#f1 score
f1_gb= (f1_score(y_test, y_pred,average='weighted'))
f1_gb

0.8763269988970728

In [ ]:
#accuracy
accuracy_gb=accuracy_score(y_test, y_pred)
accuracy_gb

0.8805194805194805

In [ ]:
scores=scores.append({'Classification Algorithm':'Gradient Boosting Classifier', 'F1 Score':f1_gb, 'Accuracy':accuracy_gb}, ignore_index=True)
scores=scores.append({'Classification Algorithm':'Logistic Regression', 'F1 Score':logreg_f1_score, 'Accuracy':logreg_accuracy}, ignore_index=True)

In [ ]:
scores=scores.sort_values('Accuracy',ascending=False)
scores.style.hide_index()

Classification Algorithm,F1 Score,Accuracy
Gradient Boosting Classifier,0.876327,0.880519
Random Forest Classifier,0.864555,0.870130
Logistic Regression,0.658683,0.851948
SVM Classifier,0.827618,0.836364
MLP Classifier,0.791605,0.815584
